In [1]:
import pandas as pd
from klse_scrapper import *
from plot_utils import *
from utils import *
from fastai.imports import *
from fastai.structured import *
from loess.loess_1d import loess_1d
from datetime import datetime, date, time, timedelta
from workalendar.asia import Malaysia
from counter import Counter
import os
from session_handler import *
from shareinvestor_scrapper import *

C:\ProgramData\Anaconda3\envs\fastai\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning:

The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.



In [2]:
topglov=Counter('SUPERMX','7106',scheduler_on=False)
topglov.set_timestamp(datetime.combine(date.today()-timedelta(days=1), time(15)).timestamp())
topglov.update_counter()
#topglov.detect_shark()

Health Care
False
1


In [5]:
topglov.update_counter()

True


In [6]:
topglov.buffer_temp

,time,buy_queue_vol,buy_queue_price,last_done_vol,last_done_price,type,sell_queue_vol,sell_queue_price
0,2020-06-20 16:55:27,None,NaN,10.0,7.4,Sell Down,None,NaN
1,2020-06-20 16:55:29,"(-71) 2,654",7.4,NaN,NaN,None,None,NaN
2,2020-06-20 16:55:29,"(-20) 2,634",7.4,NaN,NaN,None,None,NaN
3,2020-06-20 16:55:29,"(-9) 2,625",7.4,NaN,NaN,None,None,NaN
4,2020-06-20 16:55:28,None,NaN,71.0,7.4,Sell Down,None,NaN
...,...,...,...,...,...,...,...,...
143,2020-06-20 16:59:47,None,NaN,4.0,7.4,Sell Down,None,NaN
144,2020-06-20 16:59:49,"(+100) 3,586",7.4,NaN,NaN,None,None,NaN
145,2020-06-20 16:59:58,"(-5) 3,581",7.4,NaN,NaN,None,None,NaN
146,2020-06-20 16:59:58,None,NaN,5.0,7.4,Sell Down,None,NaN


In [11]:
topglov.sharks

,time,buy_vol_chg,buy_queue_price,last_done_price,price_pct_open,last_done_buy_vol,last_done_sell_vol
1035,2020-06-19 15:10:04,1000,7.29,7.29,-1.49,5727.0,2495.0
16938,2020-06-19 16:52:05,900,7.40,7.40,0.00,82572.0,46703.0
1035,2020-06-19 15:10:04,1000,7.29,7.29,-1.49,54625.0,51393.0
16938,2020-06-19 16:52:05,900,7.40,7.40,0.00,131470.0,95601.0
1035,2020-06-19 15:10:04,1000,7.29,7.29,-1.49,103523.0,100291.0
16938,2020-06-19 16:52:05,900,7.40,7.40,0.00,180368.0,144499.0


In [1]:
from ggplot import *
# Standard plotly imports
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks as cf
cf.go_offline(connected=True)
init_notebook_mode(connected=True)
from loess.loess_1d import loess_1d
import plotly.figure_factory as ff
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.offline as py

In [2]:
def split_buy_sell_queue(df_):
    def split_queue(col):
        test=df_[col].str.split(" ", n = 1, expand = True)
        buy_vol_chg=list()
        buy_vol = list()
        for item in list(test[0].fillna('0').values):
            try:
                buy_vol_chg.append(int(item.replace(',','').replace('(','').replace(')','')))
            except Exception as e:
                buy_vol_chg.append(0)
        for item in list(test[1].fillna('0').values):
            try:
                buy_vol.append(int(item.replace(',','')))
            except Exception as e:
                buy_vol.append(0)
        return buy_vol_chg,buy_vol 
    buy_vol_chg,buy_vol =split_queue('buy_queue_vol')
    sell_vol_chg,sell_vol =split_queue('sell_queue_vol')
    df_['buy_queue_vol']=buy_vol
    df_['buy_vol_chg']=buy_vol_chg
    df_['sell_queue_vol']=sell_vol
    df_['sell_vol_chg']=sell_vol_chg
    return df_

In [3]:
def construct_series(df,title):
    series={'title':title,'x':list(df.index),'y':list(df.values)}
    return series

In [4]:
def calculate_force_data(event_,interval='1min'):
    date_=datetime.fromtimestamp(event_['to']).strftime("%Y-%m-%d")
    csv_path=build_quote_movements_csv(event_)
    row={'symbol':event_['symbol'],'counter':event_['counter'],'date':date_}
    if os.path.exists(csv_path):
        df_=pd.read_csv(csv_path,parse_dates=['time'])
        df_=split_buy_sell_queue(df_)
        df_=df_.iloc[::-1]
        df_=df_.set_index('time')
        dfcal_=pd.DataFrame()
        dfcal_=df_[df_['type'].notna()].resample(interval).agg({'last_done_vol':'sum','last_done_price':'last'})
        dfcal_['price_change_pct']=dfcal_['last_done_price'].pct_change()*100
        dfcal_['force']=dfcal_['price_change_pct']*dfcal_['last_done_vol']
        row['force_sum']=dfcal_['force'].sum()
        row['force_max']=dfcal_['force'].max()
        row['force_min']=dfcal_['force'].min()
        row['force_mean']=dfcal_['force'].mean()
        row['force_std']=dfcal_['force'].std()
        row['price_change_min']=dfcal_['price_change_pct'].min()
        row['price_change_max']=dfcal_['price_change_pct'].max()
        row['price_change_std']=dfcal_['price_change_pct'].std()
        row['buy_vol_chg_max']=df_.resample(interval).agg({'buy_vol_chg':'max'})['buy_vol_chg'].max()
        row['buy_vol_chg_mean']=df_.resample(interval).agg({'buy_vol_chg':'mean'})['buy_vol_chg'].mean()
        row['sell_vol_chg_max']=df_.resample(interval).agg({'sell_vol_chg':'max'})['sell_vol_chg'].max()
        row['sell_vol_chg_mean']=df_.resample(interval).agg({'sell_vol_chg':'mean'})['sell_vol_chg'].mean()
        return row

In [5]:
def mine_force_data(board,category,from_,to_,interval):
    date_start=datetime.fromtimestamp(from_).strftime("%Y-%m-%d")
    date_end=datetime.fromtimestamp(to_).strftime("%Y-%m-%d")
    dir_=build_force_dir(category)
    if not os.path.exists(dir_): os.makedirs(dir_)   
    df=get_board_category_listings(board,category)
    lists_=df[['symbol','code','cat']].to_dict('records')
    events=build_event_lists(lists_=lists_,from_=from_,to_=to_)
    rows=[]
    for event in events:
        events_=distribute_requests(event)
        for event_ in events_:
            csv_path=build_quote_movements_csv(event_)
            if os.path.exists(csv_path):
                try:
                    row=calculate_force_data(event_,interval)
                    rows.append(row)
                except Exception as e:
                    print('Error occurred when processing: '+csv_path + '\nError: ' + str(e))
    df_=pd.DataFrame(rows)
    path=build_force_csv(category)
    #return df_
    df_.to_csv(path,index=False)

In [6]:
def bulk_update_threshold(category,interval):
    df_cal=pd.DataFrame()
    dir_='./history/calculated_data/'+ category +'/force/'
    filename=category.replace('&','').replace(' ','').lower()+'_force'
    df_=pd.read_csv(dir_+filename+'.csv',parse_dates=['date'],index_col='date')
    df_=df_.iloc[::-1]
    df_cal['buy_vol_chg_max_mean']=df_.groupby(['symbol','counter']).rolling(interval).agg({'buy_vol_chg_max':'mean'})['buy_vol_chg_max']
    df_cal['buy_vol_chg_max_std']=df_.groupby(['symbol','counter']).rolling(interval).agg({'buy_vol_chg_max':'std'})['buy_vol_chg_max']
    df_cal['sell_vol_chg_max_mean']=df_.groupby(['symbol','counter']).rolling(interval).agg({'sell_vol_chg_max':'mean'})['sell_vol_chg_max']
    df_cal['sell_vol_chg_max_std']=df_.groupby(['symbol','counter']).rolling(interval).agg({'sell_vol_chg_max':'std'})['sell_vol_chg_max']
    #df_cal['buy_vol_chg_max_mean']=df_.groupby(['symbol','counter']).resample(interval).agg({'buy_vol_chg_max':'mean'})['buy_vol_chg_max']
    #df_cal['buy_vol_chg_max_std']=df_.groupby(['symbol','counter']).resample(interval).agg({'buy_vol_chg_max':'std'})['buy_vol_chg_max']
    #df_cal['sell_vol_chg_max_mean']=df_.groupby(['symbol','counter']).resample(interval).agg({'sell_vol_chg_max':'mean'})['sell_vol_chg_max']
    #df_cal['sell_vol_chg_max_std']=df_.groupby(['symbol','counter']).resample(interval).agg({'sell_vol_chg_max':'std'})['sell_vol_chg_max']
    df_cal.reset_index().to_feather(dir_+filename)
    return df_cal

In [28]:
#Test Calculate threshold
interval=20
category='Health Care'
df_=bulk_update_threshold(category,interval)
df_=df_.reset_index()
df_[df_['counter']==7113]

,symbol,counter,date,buy_vol_chg_max_mean,buy_vol_chg_max_std,sell_vol_chg_max_mean,sell_vol_chg_max_std
275,TOPGLOV,7113,2020-05-05,NaN,NaN,NaN,NaN
276,TOPGLOV,7113,2020-05-06,NaN,NaN,NaN,NaN
277,TOPGLOV,7113,2020-05-08,NaN,NaN,NaN,NaN
278,TOPGLOV,7113,2020-05-12,NaN,NaN,NaN,NaN
279,TOPGLOV,7113,2020-05-13,NaN,NaN,NaN,NaN
280,TOPGLOV,7113,2020-05-14,NaN,NaN,NaN,NaN
281,TOPGLOV,7113,2020-05-15,NaN,NaN,NaN,NaN
282,TOPGLOV,7113,2020-05-18,NaN,NaN,NaN,NaN
283,TOPGLOV,7113,2020-05-19,NaN,NaN,NaN,NaN
284,TOPGLOV,7113,2020-05-20,NaN,NaN,NaN,NaN


In [ ]:
dir_='./history/calculated_data/'+ category +'/force/'
filename=category.replace('&','').replace(' ','').lower()+'_force'
df_=pd.read_feather(dir_+filename)
df_[df_['counter']==7113]
#print(len(df_))
#df_.iloc[-1]

In [ ]:
from messenger_handler import *
response=send_volume_alert('test')
if response.status_code==200:
    print('Successfully sent alert')

In [8]:
#event={'symbol':'COMFORT','counter':'2127','category':'Industrial Products & Services','to':1591891200}
#row=calculate_force_data(event)
#Test mine force data
%time df_=mine_force_data('Main Market','Health Care',1588554000,1592182800,'1min')

KeyboardInterrupt: 

In [ ]:
#datelist_=['2020-05-29','2020-05-28','2020-05-27']
datelist_=['2020-06-05','2020-06-04','2020-06-03','2020-06-02','2020-06-01','2020-05-29','2020-05-28','2020-05-27']
#datelist_=['2020-05-29']
df = pd.DataFrame()
for date_ in datelist_:
    df1=pd.read_csv('./history/quote_movements/Technology/INARI_0166/INARI_0166_' + date_ + '.csv')
    df=df.append(df1)
split_buy_sell_queue()
df=df.groupby('time').agg({'buy_queue_vol': 'sum',
                        'buy_queue_price': 'mean',
                        'last_done_vol':'sum', 
                        'last_done_price': 'mean',
                        'type': 'first',
                        'sell_queue_vol':'sum',
                        'sell_queue_price':'mean',
                        'buy_vol_chg':'sum',
                        'sell_vol_chg':'sum'
                       })
df['last_done_sell_down']=df[df['type']=='Sell Down'].last_done_vol
df['last_done_buy_up']=df[df['type']=='Buy Up'].last_done_vol
#df=df.set_index('time')
series=[]
series.append(construct_series(df['sell_queue_vol'],'Sell Queue Volume'))
series.append(construct_series(df['buy_queue_vol'],'Buy Queue Volume'))
series.append(construct_series(df['last_done_price'],'Lost Done Price'))
plot_multi_axes(series,'Queue Volume VS Last Done Price')
series1=[]
series1.append(construct_series(df['last_done_sell_down'],'Sell Down Volume'))
series1.append(construct_series(df['last_done_buy_up'],'Buy Up Volume'))
series1.append(construct_series(df['last_done_price'],'Lost Done Price'))
plot_multi_axes(series1,'Last Done Volume VS Last Done Price')
series2=[]
series2.append(construct_series(df['sell_queue_vol'],'Sell Queue Volume'))
series2.append(construct_series(df['buy_queue_vol'],'Buy Queue Volume'))
series2.append(construct_series(df['last_done_price'],'Last Done Price'))
plot_multi_axes(series2,'Last Done Volume VS Last Done Price')
series2=[]
series2.append(construct_series(df['sell_queue_vol'],'Sell Queue Volume'))
series2.append(construct_series(df['buy_queue_vol'],'Buy Queue Volume'))
series2.append(construct_series(df['buy_queue_price'],'Buy Queue Price'))
plot_multi_axes(series2,'Last Done Volume VS Buy Queue Price')

In [ ]:
#Case Study: Analyze buy and sell queue in bull market
interval='10min'
#cat='Industrial Products & Services'
cat='Energy'
symbol='HIBISCS'
counter='5199'
name_=symbol+'_'+counter
date_='2020-06-10'
df=pd.read_csv('./history/quote_movements/'+ cat +'/'+name_+'/'+name_ + '_' + date_ + '.csv',parse_dates=['time'])
df['hour']=df['time'].transform(lambda x: x.hour)
df['minute']=df['time'].transform(lambda x: 5*(x.minute//5))
df=split_buy_sell_queue(df)
df=df.iloc[::-1]
#df1=df.groupby(['hour','minute','sell_queue_price']).agg({'sell_vol_chg':'sum','sell_queue_vol':'last'}).reset_index()
df1=df.set_index('time')
df_agg=pd.DataFrame()
df_agg1=pd.DataFrame()
#for sellqp in df1['sell_queue_price'].dropna().unique():
#    df_resampled=pd.DataFrame()
#    df_resampled1=pd.DataFrame()
    #group by sell queue price
    #df_resampled_=df1[df1['sell_queue_price']==sellqp].resample(interval).agg({'sell_vol_chg':'max','sell_queue_vol':'last'}).dropna()
#    df_resampled_=df1.resample(interval).agg({'sell_vol_chg':'max','sell_queue_vol':'last'}).dropna()
#    df_resampled[str(sellqp)+'_sell_queue_vol']=df_resampled_['sell_queue_vol']
#    df_resampled1[str(sellqp)+'_sell_vol_chg']=df_resampled_['sell_vol_chg']
#    df_agg=df_agg.append(df_resampled)
#    df_agg1=df_agg1.append(df_resampled1)
df_resampled=pd.DataFrame()
df_resampled1=pd.DataFrame()
df_resampled_=df1.resample(interval).agg({'sell_vol_chg':'max','sell_queue_vol':'last'}).dropna()
df_resampled['sell_queue_vol']=df_resampled_['sell_queue_vol']
df_resampled1['sell_vol_chg']=df_resampled_['sell_vol_chg']
df_agg=df_agg.append(df_resampled)
df_agg1=df_agg1.append(df_resampled1)
df_agg.iplot()
df_agg1.iplot()

df_ldp=df1.resample(interval).agg({'last_done_price':'last'})
df_ldp['price_change_rate']=df_ldp['last_done_price'].pct_change()*100
df_ldp.iplot()

#Last done volume by type (Sell Down & Buy Up)
df_ldv=pd.DataFrame()
df_resampled=pd.DataFrame()
df_resampled_=df1[df1['type']=='Sell Down'].resample(interval).agg({'last_done_vol':'sum','last_done_price':'last'})
df_resampled['Sell Down']=df_resampled_['last_done_vol']
df_ldv=df_ldv.append(df_resampled)
df_resampled=pd.DataFrame()
df_resampled_=df1[df1['type']=='Buy Up'].resample(interval).agg({'last_done_vol':'sum','last_done_price':'last'})
df_resampled['Buy Up']=df_resampled_['last_done_vol']
df_ldv=df_ldv.append(df_resampled)
df_ldv.iplot()

#Last done volume 
df_resampled_=pd.DataFrame()
df_resampled_=df1[df1['type'].notna()].resample(interval).agg({'last_done_vol':'sum','last_done_price':'last'})
df_resampled_['price_change_pct']=df_resampled_['last_done_price'].pct_change()*100
df_resampled_['force']=df_resampled_['price_change_pct']*df_resampled_['last_done_vol']
series=[]
series.append(construct_series(df_resampled_['force'],'Force'))
series.append(construct_series(df_resampled_['force'],'Force'))
series.append(construct_series(df_resampled_['price_change_pct'],'Price Change Percentage'))
plot_multi_axes(series,'Force VS Price Change Percentage')
total_force=df_resampled_['force'].sum()
print(total_force)
#df_resampled_.iplot()

df_agg=pd.DataFrame()
df_agg1=pd.DataFrame()
#for sellqp in df1['buy_queue_price'].dropna().unique():
#    df_resampled=pd.DataFrame()
#    df_resampled1=pd.DataFrame()
#    df_resampled_=df1[df1['buy_queue_price']==sellqp].resample(interval).agg({'buy_vol_chg':'max','buy_queue_vol':'last'}).dropna()
#    df_resampled[str(sellqp)+'_buy_queue_vol']=df_resampled_['buy_queue_vol']
#    df_resampled1[str(sellqp)+'_buy_vol_chg']=df_resampled_['buy_vol_chg']
#    df_agg=df_agg.append(df_resampled)
#    df_agg1=df_agg1.append(df_resampled1)
df_resampled=pd.DataFrame()
df_resampled1=pd.DataFrame()
df_resampled_=df1.resample(interval).agg({'buy_vol_chg':'max','buy_queue_vol':'last'}).dropna()
df_resampled['buy_queue_vol']=df_resampled_['buy_queue_vol']
df_resampled1['buy_vol_chg']=df_resampled_['buy_vol_chg']
df_agg=df_agg.append(df_resampled)
df_agg1=df_agg1.append(df_resampled1)
df_agg.iplot()
df_agg1.iplot()

In [11]:
generate_session()

In [2]:
topglov=Counter('SUPERMX','7106',scheduler_on=False)
topglov.set_timestamp(datetime.combine(date.today()-timedelta(days=2), time(9)).timestamp())
topglov.get_day_volume()
topglov.detect_shark()

Health Care
False


In [3]:
topglov.sharks

,time,buy_vol_chg,buy_queue_price,last_done_price,price_pct_open,last_done_buy_vol,last_done_sell_vol
1643,2020-06-19 09:05:10,2130.0,6.85,6.88,-0.72,8980.0,3647.0
1863,2020-06-19 09:06:09,2594.0,6.86,6.87,-0.87,10082.0,4324.0
7373,2020-06-19 09:38:00,1326.0,6.89,6.89,-0.58,41234.0,23047.0
9756,2020-06-19 10:03:04,1500.0,7.00,7.00,1.01,56200.0,29421.0
9778,2020-06-19 10:03:06,1000.0,7.00,7.01,1.15,56447.0,29421.0
10858,2020-06-19 10:13:45,1000.0,6.99,6.99,0.87,61334.0,34474.0
18382,2020-06-19 12:07:21,1108.0,7.04,7.04,1.59,92666.0,55573.0
18495,2020-06-19 12:08:24,1297.0,7.06,7.06,1.88,93517.0,56046.0
25580,2020-06-19 14:42:00,1000.0,7.28,7.28,5.05,137172.0,75676.0
29435,2020-06-19 15:10:04,1000.0,7.29,7.29,5.19,155131.0,88657.0


In [42]:
interval='5min'
df_=pd.DataFrame()
df_=topglov.buffer_temp
df_['time']=pd.to_datetime(df_['time'])
df_=df_.set_index('time')
df_.resample(interval).agg({'last_done_vol':'sum'}).cumsum()

,last_done_vol
time,
2020-06-19 06:05:00,0.0
2020-06-19 06:10:00,0.0
2020-06-19 06:15:00,0.0
2020-06-19 06:20:00,0.0
2020-06-19 06:25:00,0.0
...,...
2020-06-19 16:35:00,349614.0
2020-06-19 16:40:00,357858.0
2020-06-19 16:45:00,357858.0


In [39]:
interval='1min'
df_=pd.DataFrame()
df_=topglov.buffer_temp
df_=split_buy_sell_queue(df_)
df_=df_.set_index('time')
df_['last_done_price']=df_['last_done_price'].fillna(method='ffill').fillna(topglov.today_open)
#df_['']=df_.resample(interval).agg({'last_done_price':'last'}).dropna().pct_change()
df__=pd.DataFrame()
df__=df__.append(topglov.sharks.set_index('time').resample(interval).agg({'buy_vol_chg':'count'}))
df__['last_done_price_diff']=df_.resample(interval).agg({'last_done_price':'last'}).dropna().diff()
df__['last_done_price_pct']=df_.resample(interval).agg({'last_done_price':'last'}).dropna().pct_change()
df__['last_done_vol_sum']=df_.resample(interval).agg({'last_done_vol':'sum'}).cumsum()
df__['last_done_price_pct']=round(df__['last_done_price_pct']*100,2)
df__['last_done_price']=df_.resample(interval).agg({'last_done_price':'last'})
df__['last_open_price_diff']=df__['last_done_price']-topglov.today_open
df__['last_open_price_pct']=df__['last_done_price'].transform(lambda x: round((x - topglov.today_open)/topglov.today_open * 100,2))
df__=df__.fillna(method='ffill')

In [25]:
df_=topglov.buffer_temp.copy()
df_[['last_done_vol','last_done_price','type']]=df_[['last_done_vol','last_done_price','type']].fillna(method='ffill')
df_['last_done_price']=df_['last_done_price'].fillna(topglov.today_open)
df_['last_done_vol']=df_['last_done_vol'].fillna(0)
df_['buy_queue_price_diff']=df_['buy_queue_price']-df_['last_done_price']
df_['sell_queue_price_diff']=df_['sell_queue_price']-df_['last_done_price']
df_['buy_queue_type']=df_['buy_queue_price_diff'].dropna().transform(lambda x: 'Buy Up' if (x > 0) else 'Buy Down')
df_['sell_queue_type']=df_['sell_queue_price_diff'].dropna().transform(lambda x: 'Sell Up' if (x > 0) else 'Sell Down')

In [26]:
df_['buy_queue_price'][df_['buy_queue_price'].isna()]=df_['sell_queue_price'][df_['sell_queue_price'].notna()]
df_['buy_queue_type'][df_['buy_queue_type'].isna()]=df_['sell_queue_type'][df_['sell_queue_type'].notna()]
df_['buy_vol_chg']=df_['sell_vol_chg'][df_['sell_vol_chg']>0]
df_=df_.rename(columns={'buy_queue_price':'queue_price','buy_queue_type':'queue_type','buy_vol_chg':'vol_chg'})
df_=df_.drop(columns=['sell_vol_chg','sell_queue_type','sell_queue_price_diff','buy_queue_price_diff'])
df__=df_[['time','queue_price','vol_chg','queue_type','last_done_price']].dropna().copy()

C:\ProgramData\Anaconda3\envs\fastai\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\envs\fastai\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
df_

,time,buy_queue_vol,queue_price,last_done_vol,last_done_price,type,sell_queue_vol,sell_queue_price,vol_chg,queue_type
0,2020-06-19 06:05:14,66.0,6.93,0.0,7.4,NaN,NaN,NaN,NaN,Buy Down
1,2020-06-19 06:05:14,NaN,6.98,0.0,7.4,NaN,30.0,6.98,NaN,Sell Down
2,2020-06-19 08:30:00,NaN,6.75,0.0,7.4,NaN,14.0,6.75,NaN,Sell Down
3,2020-06-19 08:30:00,11.0,6.98,0.0,7.4,NaN,NaN,NaN,NaN,Buy Down
4,2020-06-19 08:30:01,16.0,6.98,0.0,7.4,NaN,NaN,NaN,NaN,Buy Down
...,...,...,...,...,...,...,...,...,...,...
45658,2020-06-19 16:59:47,NaN,NaN,4.0,7.4,Sell Down,NaN,NaN,NaN,NaN
45659,2020-06-19 16:59:49,3586.0,7.40,4.0,7.4,Sell Down,NaN,NaN,NaN,Buy Down
45660,2020-06-19 16:59:58,3581.0,7.40,4.0,7.4,Sell Down,NaN,NaN,NaN,Buy Down
45661,2020-06-19 16:59:58,NaN,NaN,5.0,7.4,Sell Down,NaN,NaN,NaN,NaN


In [54]:
topglov.sharks.index.tolist()

[12445, 24764, 24520, 24397, 8009, 7562]

In [57]:
topglov.sharks['last_done_price']=df_.reset_index().iloc[topglov.sharks.index.tolist()]['last_done_price']

In [9]:
topglov.sharks

,time,buy_vol_chg,buy_queue_price,last_done_price
12445,2020-06-17 11:14:06,2000,7.08,NaN
24764,2020-06-17 09:04:15,1000,7.44,NaN
24520,2020-06-17 09:05:29,1000,7.46,NaN
24397,2020-06-17 09:06:06,1000,7.46,NaN
8009,2020-06-17 14:42:51,1000,7.21,NaN
7562,2020-06-17 14:46:57,1000,7.25,NaN


In [47]:
df_=pd.read_csv('history\quote_movements\Health Care\SUPERMX_7106\SUPERMX_7106_2020-06-15.csv')

In [59]:
last_done_group=df_.groupby('type').agg({'last_done_vol':'sum'})

In [68]:
last_done_group.loc['Sell Down'].last_done_vol


340981.0

In [25]:
df_=topglov.buffer.iloc[::-1]
df_=split_buy_sell_queue(df_)
last_done_df=df_[['time','last_done_vol','last_done_price','type']]
last_done_df=last_done_df.iloc[::-1]
last_done=pd.DataFrame()
last_done['last_done_price']=last_done_df['last_done_price'].fillna(method='ffill').fillna(123)
last_done['last_done_sell_vol']=last_done_df[last_done_df['type']=='Sell Down'].last_done_vol.cumsum()
last_done['last_done_sell_vol']=last_done['last_done_sell_vol'].fillna(method='ffill').fillna(0)
last_done['last_done_buy_vol']=last_done_df[last_done_df['type']=='Buy Up'].last_done_vol.cumsum()
last_done['last_done_buy_vol']=last_done['last_done_buy_vol'].fillna(method='ffill').fillna(0)

In [26]:
last_done

,last_done_price,last_done_sell_vol,last_done_buy_vol
25765,123.00,0.0,0.0
25764,123.00,0.0,0.0
25763,123.00,0.0,0.0
25762,123.00,0.0,0.0
25761,123.00,0.0,0.0
...,...,...,...
4,7.21,86505.0,66360.0
3,7.21,86505.0,66360.0
2,7.21,86505.0,66369.0
1,7.21,86505.0,66379.0
